### Notebook show work of GoogLeNet model

In [1]:
import sys

import numpy as np
import tensorflow as tf
from pandas import ewma
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqn
from IPython.display import clear_output
%matplotlib inline

sys.path.append('..')
from inception import InceptionV1

from dataset.dataset.opensets import MNIST
from dataset import V,B,F

In [2]:
mnist = MNIST()

Extracting ExtractingExtractingExtracting/tmp/train-labels-idx1-ubyte.gz   
/tmp/t10k-labels-idx1-ubyte.gz/tmp/t10k-images-idx3-ubyte.gz/tmp/train-images-idx3-ubyte.gz




In [3]:
placeholders_config = {
                       'images': {'shape': (28, 28, 1),
                                 'dtype': 'float32',
                                 'data_format': 'channels_last'},
                
                       'labels': {'shape': (10),
                                 'dtype': 'int32',
                                 'data_format': 'channels_last',
                                 'transform': 'ohe',
                                 'name': 'targets'}
}

In [4]:
model_config = {'inputs': placeholders_config,
               'loss':'softmax_cross_entropy',
               'optimizer':'Adam',
               'dim': 2,
               'data_format':'channels_last'}

In [5]:
train_dict = {'images': B('images'),
              'labels': B('labels')}

In [6]:
pipeline = (mnist.train
           .p
           .init_variable('loss', init_on_each_run=list)
           .init_model('dynamic',
                      InceptionV1,
                      'googlenet',
                      config=model_config)
           .train_model('googlenet',
                        fetches='loss',
                        feed_dict=train_dict,
                        save_to=V('loss'), mode='a'))

In [7]:
for i in tqn(range(500)):
    pipeline.next_batch(200, n_epochs=None, shuffle=True)
    
    if i%10==0:
        clear_output(True)
        plt.plot(ewma(np.array(pipeline.get_variable('loss')[-100:]),span=10,adjust=False))
        plt.show()

AttributeError: 'dict' object has no attribute 'dtype'

In [ ]:
predict = (mnist.test
          .p
          .init_variable('prediction', init_on_each_run=list)
          .init_variable('image', init_on_each_run=list)
          .init_variable('true_value', init_on_each_run=list)
          .import_model('googlenet', pipeline)
          .predict_model('googlenet',
                         fetches=['images', 'labels_hat', 'labels'],
                         feed_dict=train_dict,
                         save_to=[V('image'), V('prediction'), V('true_value')],
                         mode='a')).run(1, n_epochs=1)

In [ ]:
images = predict.get_variable('image')
prediction = predict.get_variable('prediction')
true_value = predict.get_variable('true_value')

_, ax = plt.subplots(3, 4, figsize=(16, 16))
ax = ax.reshape(-1)
plot = 0
for i in np.random.randint(0, len(true_value), 12):
    ax[plot].imshow(images[i].reshape(-1,28))
    ax[plot].set_xlabel('Predict: {}, True: {}'.format(int(prediction[i][0]), int(true_value[i][0])), fontsize=18)
    plot += 1